# DAA Experiment Harness (Template)
**Tujuan**: kerangka eksperimen untuk membandingkan *Algoritma A* vs *Algoritma B* pada instance unik per kelompok.
1. Isi `algo_A` dan `algo_B`.
2. Sesuaikan `generate_instances` atau loader data.
3. Atur `Ns`, `repeats`, `seed`.


In [4]:
import os, time, random, statistics
from pathlib import Path
import numpy as np, pandas as pd
import matplotlib.pyplot as plt

Path('results').mkdir(exist_ok=True)

random.seed(42)
np.random.seed(42)

## Data Aquisition

In [5]:
##LOAD DATA

import pandas as pd

df = pd.read_csv(
    r"E:\DAA_Kelompok4_KelasA\data\normalized_fix.csv"
)

df.head(10)

,Hari,Jam,Mata Kuliah,Semester,SKS,Dosen,Ruang,Kelas,Kapasitas Kelas,start,finish,duration
0,Senin,09:20:00 - 10:10:00,Kalkulus I,1,3,SUPRIYADI WIBOWO,R001,A,25,09:20:00,10:10:00,50.0
1,Senin,10:15:00 - 11:05:00,Kalkulus I,1,3,SUPRIYADI WIBOWO,R001,A,25,10:15:00,11:05:00,50.0
2,Senin,11:10:00 - 12:00:00,Kalkulus I,1,3,SUPRIYADI WIBOWO,R001,A,25,11:10:00,12:00:00,50.0
3,Senin,13:00:00 - 13:50:00,Pengolahan Citra Digital,5,3,HERI PRASETYO,R001,B,25,13:00:00,13:50:00,50.0
4,Senin,13:55:00 - 14:45:00,Pengolahan Citra Digital,5,3,HERI PRASETYO,R001,B,25,13:55:00,14:45:00,50.0
5,Senin,15:30:00 - 16:20:00,Pengolahan Citra Digital,5,3,HERI PRASETYO,R001,B,25,15:30:00,16:20:00,50.0
6,Senin,16:25:00 - 17:15:00,Data Mining,5,3,WIRANTO,R001,E,25,16:25:00,17:15:00,50.0
7,Senin,18:10:00 - 19:00:00,Data Mining,5,3,WIRANTO,R001,E,25,18:10:00,19:00:00,50.0
8,Senin,22:15:00 - 23:05:00,Data Mining,5,3,WIRANTO,R001,E,25,22:15:00,23:05:00,50.0
9,Senin,13:00:00 - 13:50:00,Bahasa Indonesia,1,2,KUNDHARU SADDHONO,R002,C,40,13:00:00,13:50:00,50.0


In [6]:
##MENAMBAHKAN VARIABEL ACAK PADA KOLOM JUMLAH MAHASISWA
import pandas as pd
import numpy as np

# Set seed untuk reproducibility
np.random.seed(42)                                              # O(1)

# Load dataset
df = pd.read_csv(r"E:\DAA_Kelompok4_KelasA\data\normalized_fix.csv")  # O(n)

# Tambahkan kolom jumlah mahasiswa berdasarkan kapasitas kelas
df['jumlah_mahasiswa'] = df['Kapasitas Kelas'].apply(           # O(n)
    lambda cap: np.random.randint(int(cap*0.4), cap+1)         # O(1)
)


In [7]:
# Simpan kembali dataset baru
df.to_csv(r"E:\DAA_Kelompok4_KelasA\data\normalized_with_students.csv", index=False)   # O(n)

## Implementasi Algoritma A (Greedy EFT)

In [8]:
import pandas as pd
import time

# ============================================================
# LOAD DATASET
# ============================================================

def load_intervals(path):
    df = pd.read_csv(path)                                   # O(n)

    # Konversi start & finish ke datetime.time
    df['start'] = pd.to_datetime(df['start']).dt.time        # O(n)
    df['finish'] = pd.to_datetime(df['finish']).dt.time      # O(n)

    return df                                                # O(1)

In [9]:
# ============================================================
# ALGORITMA A — GREEDY EARLIEST FINISH TIME (EFT)
# ============================================================

def greedy_earliest_finish(intervals):
    # Ubah agar time comparison valid
    intervals['start_dt'] = pd.to_datetime(intervals['start'].astype(str))   # O(n)
    intervals['finish_dt'] = pd.to_datetime(intervals['finish'].astype(str)) # O(n)

    # Sort berdasarkan waktu selesai (finish) → langkah inti greedy
    sorted_idx = intervals.sort_values('finish_dt').index.tolist()           # O(n log n)

    selected = []                                                            # O(1)
    last_finish = pd.to_datetime("00:00")                                    # O(1)

    # Loop memilih interval
    for idx in sorted_idx:                                                   # O(n)
        start_val = intervals.loc[idx, 'start_dt']                           # O(1)
        finish_val = intervals.loc[idx, 'finish_dt']                         # O(1)

        if start_val >= last_finish:                                         # O(1)
            selected.append(idx)                                             # O(1)
            last_finish = finish_val                                         # O(1)

    return selected                                                          # O(k)

In [10]:
# ============================================================
# RUNNER UNTUK MENJALANKAN ALGORITMA A
# ============================================================

def run_baseline_A():
    path = r"E:\DAA_Kelompok4_KelasA\data\normalized_fix.csv"   # O(1)
    intervals = load_intervals(path)                            # O(n)

    t0 = time.time()                                            # O(1)
    selected = greedy_earliest_finish(intervals)                # O(n log n)
    t1 = time.time()                                            # O(1)

    exec_time = (t1 - t0) * 1000                                # O(1)

    print("Jumlah interval terpilih:", len(selected))
    print("Waktu eksekusi (ms):", exec_time)

    return selected, exec_time                                  # O(1)

In [11]:
##KASUS KECIL

import pandas as pd

test_df = pd.DataFrame({
    'start':  ["09:00", "09:30", "10:00"],
    'finish': ["09:50", "10:20", "11:00"],
    'duration': [50, 50, 60],
    'profit': [10, 5, 8]
})

In [12]:
selected, exec_time = greedy_earliest_finish(test_df), 0
print(selected)

[0, 2]


C:\Users\Asus\AppData\Local\Temp\ipykernel_30488\1179081887.py:7: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  intervals['start_dt'] = pd.to_datetime(intervals['start'].astype(str))   # O(n)
C:\Users\Asus\AppData\Local\Temp\ipykernel_30488\1179081887.py:8: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  intervals['finish_dt'] = pd.to_datetime(intervals['finish'].astype(str)) # O(n)


In [13]:
##MENGGUNAKAN NON OVERLAP
test_df = pd.DataFrame({
    'start':  ["08:00", "09:00", "10:00", "11:00"],
    'finish': ["09:00", "10:00", "11:00", "12:00"],
    'duration': [60, 60, 60, 60],
    'profit': [10, 20, 30, 40]
})

selected = greedy_earliest_finish(test_df)
print(selected)

[0, 1, 2, 3]


C:\Users\Asus\AppData\Local\Temp\ipykernel_30488\1179081887.py:7: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  intervals['start_dt'] = pd.to_datetime(intervals['start'].astype(str))   # O(n)
C:\Users\Asus\AppData\Local\Temp\ipykernel_30488\1179081887.py:8: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  intervals['finish_dt'] = pd.to_datetime(intervals['finish'].astype(str)) # O(n)


## Implementasi Algoritma (Greedy Profit Density)

In [14]:
import pandas as pd

def greedy_profit_density(intervals):
    
    # --------------------------------------------------------
    # 1. Copy dataset (agar tidak mengubah data asli)
    # --------------------------------------------------------
    data = intervals.copy()                     # O(n)

    # --------------------------------------------------------
    # 2. Hitung profit density = profit / duration
    # --------------------------------------------------------
    data['density'] = data['profit'] / data['duration']   # O(n)

    # --------------------------------------------------------
    # 3. Urutkan interval berdasarkan density (descending)
    # --------------------------------------------------------
    sorted_idx = (
        data
        .sort_values('density', ascending=False)           # O(n log n)
        .index
        .tolist()
    )

    # --------------------------------------------------------
    # 4. Proses seleksi interval (cek overlap)
    # --------------------------------------------------------
    selected = []                                          # O(1)
    last_finish = pd.to_datetime("1900-01-01 00:00")       # O(1)

    for idx in sorted_idx:                                 # O(n)
        start = data.loc[idx, 'start_dt']                  # O(1)
        finish = data.loc[idx, 'finish_dt']                # O(1)

        if start >= last_finish:                           # O(1)
            selected.append(idx)                           # O(1)
            last_finish = finish                           # O(1)

    # --------------------------------------------------------
    # 5. Kembalikan hasil
    # --------------------------------------------------------
    return selected                                        # O(k)


In [20]:
df = load_instance("../data/normalized_fix.csv").head(20)

res_B = greedy_profit_density(df)
print("Interval terpilih (B):", res_B)
print("Jumlah interval:", len(res_B))


Interval terpilih (B): [12, 9, 11, 8]
Jumlah interval: 4


In [ ]:
## PILOT RUN
import time

df_full = load_instance("../data/normalized_fix.csv")

Ns = [10, 20, 40]   # kecil – sedang

for n in Ns:
    df_n = df_full.head(n)

    # Algoritma A (EFT)
    t0 = time.time()
    res_A = greedy_earliest_finish(df_n)
    t1 = time.time()

    # Algoritma B (Profit Density)
    t2 = time.time()
    res_B = greedy_profit_density(df_n)
    t3 = time.time()

    print(f"\nN = {n}")
    print(f"EFT     → intervals: {len(res_A)}, time: {(t1-t0)*1000:.3f} ms")
    print(f"Density → intervals: {len(res_B)}, time: {(t3-t2)*1000:.3f} ms")



N = 10
EFT     → intervals: 9, time: 14.179 ms
Density → intervals: 6, time: 10.783 ms

N = 20
EFT     → intervals: 11, time: 12.079 ms
Density → intervals: 4, time: 5.826 ms

N = 40
EFT     → intervals: 11, time: 12.895 ms
Density → intervals: 6, time: 6.035 ms


C:\Users\Asus\AppData\Local\Temp\ipykernel_32172\1179081887.py:7: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  intervals['start_dt'] = pd.to_datetime(intervals['start'].astype(str))   # O(n)
C:\Users\Asus\AppData\Local\Temp\ipykernel_32172\1179081887.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  intervals['start_dt'] = pd.to_datetime(intervals['start'].astype(str))   # O(n)
C:\Users\Asus\AppData\Local\Temp\ipykernel_32172\1179081887.py:8: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please 

## Pembangkit/Loader Instance (SESUAIKAN)

In [16]:
import sys                     # O(1)
import os                      # O(1)

# Menambahkan path folder ../src ke Python path
sys.path.append(               # O(1) 
    os.path.abspath("../src")  # O(1) 
)

import loader                  # O(1)
print(dir(loader))             # O(m)


['__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__spec__', 'load_instance', 'pd']


In [17]:
from loader import load_instance #O(1)
df = load_instance("../data/normalized_fix.csv") #O(n)
df.head(10) #O(1)


,Hari,Jam,Mata Kuliah,Semester,SKS,Dosen,Ruang,Kelas,Kapasitas Kelas,start,finish,duration,start_dt,finish_dt,profit
0,Senin,09:20:00 - 10:10:00,Kalkulus I,1,3,SUPRIYADI WIBOWO,R001,A,25,09:20:00,10:10:00,50.0,2025-12-12 09:20:00,2025-12-12 10:10:00,25
1,Senin,10:15:00 - 11:05:00,Kalkulus I,1,3,SUPRIYADI WIBOWO,R001,A,25,10:15:00,11:05:00,50.0,2025-12-12 10:15:00,2025-12-12 11:05:00,25
2,Senin,11:10:00 - 12:00:00,Kalkulus I,1,3,SUPRIYADI WIBOWO,R001,A,25,11:10:00,12:00:00,50.0,2025-12-12 11:10:00,2025-12-12 12:00:00,25
3,Senin,13:00:00 - 13:50:00,Pengolahan Citra Digital,5,3,HERI PRASETYO,R001,B,25,13:00:00,13:50:00,50.0,2025-12-12 13:00:00,2025-12-12 13:50:00,25
4,Senin,13:55:00 - 14:45:00,Pengolahan Citra Digital,5,3,HERI PRASETYO,R001,B,25,13:55:00,14:45:00,50.0,2025-12-12 13:55:00,2025-12-12 14:45:00,25
5,Senin,15:30:00 - 16:20:00,Pengolahan Citra Digital,5,3,HERI PRASETYO,R001,B,25,15:30:00,16:20:00,50.0,2025-12-12 15:30:00,2025-12-12 16:20:00,25
6,Senin,16:25:00 - 17:15:00,Data Mining,5,3,WIRANTO,R001,E,25,16:25:00,17:15:00,50.0,2025-12-12 16:25:00,2025-12-12 17:15:00,25
7,Senin,18:10:00 - 19:00:00,Data Mining,5,3,WIRANTO,R001,E,25,18:10:00,19:00:00,50.0,2025-12-12 18:10:00,2025-12-12 19:00:00,25
8,Senin,22:15:00 - 23:05:00,Data Mining,5,3,WIRANTO,R001,E,25,22:15:00,23:05:00,50.0,2025-12-12 22:15:00,2025-12-12 23:05:00,25
9,Senin,13:00:00 - 13:50:00,Bahasa Indonesia,1,2,KUNDHARU SADDHONO,R002,C,40,13:00:00,13:50:00,50.0,2025-12-12 13:00:00,2025-12-12 13:50:00,40


In [18]:
import pandas as pd

def test_eft_overlap():
    # Dataset kecil (manual)
    df_test = pd.DataFrame({
        'start':  ['09:00', '09:30', '10:00'],
        'finish': ['09:50', '10:20', '11:00'],
        'duration': [50, 50, 60],
        'profit': [10, 5, 8]
    })

    # Konversi waktu (samakan dengan loader)
    df_test['start_dt']  = pd.to_datetime(df_test['start'], format="%H:%M")
    df_test['finish_dt'] = pd.to_datetime(df_test['finish'], format="%H:%M")

    # Jalankan algoritma EFT
    result = greedy_earliest_finish(df_test)

    # Hasil yang BENAR secara teori
    expected = [0, 2]

    assert result == expected, f"Expected {expected}, got {result}"
    print("✅ Test EFT (overlap) LULUS")


In [19]:
test_eft_overlap()


✅ Test EFT (overlap) LULUS


C:\Users\Asus\AppData\Local\Temp\ipykernel_30488\1179081887.py:7: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  intervals['start_dt'] = pd.to_datetime(intervals['start'].astype(str))   # O(n)
C:\Users\Asus\AppData\Local\Temp\ipykernel_30488\1179081887.py:8: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  intervals['finish_dt'] = pd.to_datetime(intervals['finish'].astype(str)) # O(n)


## Evaluator & Timing

In [21]:
def run_once(algorithm, instance):
    """
    Jalankan satu algoritma pada satu instance
    Return:
    - waktu eksekusi (ms)
    - gap / feasibility (0 = feasible, 1 = infeasible)
    """
    t0 = time.perf_counter()                       # O(1)
    out = algorithm(instance)                      # O(T(n)) tergantung algoritma
    dt = (time.perf_counter() - t0) * 1000.0       # O(1)

    gap = evaluate_solution(out, instance)         # O(k) k = jumlah interval terpilih

    return dt, gap


In [22]:
def run_once(algorithm, instance):
    """
    Jalankan satu algoritma pada satu instance
    Return:
    - waktu eksekusi (ms)
    - gap (0 = feasible, 1 = infeasible)
    """
    t0 = time.perf_counter()                       # O(1)
    out = algorithm(instance)                      # O(T(n))
    dt = (time.perf_counter() - t0) * 1000.0       # O(1)

    gap = evaluate_solution(out, instance)         # O(k)

    return dt, gap


In [24]:
def greedy_earliest_finish(intervals):
    sorted_idx = (
        intervals
        .sort_values('finish_dt')
        .index
        .tolist()
    )

    selected = []
    last_finish = pd.to_datetime("1900-01-01 00:00")

    for idx in sorted_idx:
        start = intervals.loc[idx, 'start_dt']
        finish = intervals.loc[idx, 'finish_dt']

        if start >= last_finish:
            selected.append(idx)
            last_finish = finish

    return selected


In [26]:
def evaluate_solution(selected_idx, df):
    if not isinstance(selected_idx, list):
        return 1.0

    intervals = df.loc[selected_idx].sort_values('start_dt')

    last_finish = None
    for _, row in intervals.iterrows():
        if last_finish is not None and row['start_dt'] < last_finish:
            return 1.0
        last_finish = row['finish_dt']

    return 0.0


In [27]:
dt_A, gap_A = run_once(greedy_earliest_finish, df)
dt_B, gap_B = run_once(greedy_profit_density, df)

print("EFT  → time:", dt_A, "gap:", gap_A)
print("B    → time:", dt_B, "gap:", gap_B)


EFT  → time: 5.9125000043422915 gap: 0.0
B    → time: 6.0611000008066185 gap: 0.0


## Eksekusi Eksperimen (atur parameter)

In [29]:
# Load dataset utama
df_full = load_instance("../data/normalized_fix.csv")

In [30]:
# ============================================================
# PARAMETER EKSPERIMEN (HARI 6)
# ============================================================

Ns = [10, 20, 40]          # ukuran masalah
seeds = [1, 2, 3, 4, 5]    # seed untuk reproducibility
repeats = 3               # jumlah pengulangan

algorithms = {
    "EFT": greedy_earliest_finish,
    "Density": greedy_profit_density
}

In [ ]:
# ============================================================
# EKSEKUSI EKSPERIMEN (BATCH RUN)
# ============================================================

for n in Ns:
    print("=" * 50)
    print(f"UKURAN MASALAH (n) = {n}")
    print("=" * 50)

    for alg_name, alg_func in algorithms.items():
        times = []
        gaps = []
        counts = []

        for seed in seeds:
            # set seed (reproducibility)
            np.random.seed(seed)

            # sampling / shuffle data
            df_n = df_full.sample(n=n, random_state=seed)

            for r in range(repeats):
                dt, gap = run_once(alg_func, df_n)

                times.append(dt)
                gaps.append(gap)
                counts.append(len(alg_func(df_n)))

        # Hitung statistik
        mean_time = np.mean(times)
        std_time = np.std(times)

        mean_gap = np.mean(gaps)
        mean_count = np.mean(counts)

        # Print ringkasan
        print(f"\nAlgoritma: {alg_name}")
        print(f"Rata-rata waktu (ms): {mean_time:.3f} ± {std_time:.3f}")
        print(f"Rata-rata gap       : {mean_gap:.2f}")
        print(f"Rata-rata interval  : {mean_count:.2f}")

    print("\n")


UKURAN MASALAH (n) = 10

Algoritma: EFT
Rata-rata waktu (ms): 3.801 ± 1.095
Rata-rata gap       : 0.00
Rata-rata interval  : 5.80

Algoritma: Density
Rata-rata waktu (ms): 4.785 ± 0.530
Rata-rata gap       : 0.00
Rata-rata interval  : 3.40


UKURAN MASALAH (n) = 20

Algoritma: EFT
Rata-rata waktu (ms): 3.715 ± 0.463
Rata-rata gap       : 0.00
Rata-rata interval  : 8.00

Algoritma: Density
Rata-rata waktu (ms): 7.641 ± 8.153
Rata-rata gap       : 0.00
Rata-rata interval  : 2.80


UKURAN MASALAH (n) = 40

Algoritma: EFT
Rata-rata waktu (ms): 5.781 ± 1.242
Rata-rata gap       : 0.00
Rata-rata interval  : 10.20

Algoritma: Density
Rata-rata waktu (ms): 6.134 ± 0.965
Rata-rata gap       : 0.00
Rata-rata interval  : 3.60




: 

## Plot & Tabel

## (Opsional) Uji Statistik (paired t-test)

## Ekspor tabel LaTeX (opsional)